### Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re

In [ ]:
data = pd.read_csv('/datasets/real_estate_data.csv', sep = '\t')
data.info()
data.hist(figsize = (15,20))
plt.show()
data.head(10)
pd.options.mode.chained_assignment = None

Предстоит обработать и проанализировать датафрейм(data), состоящий из 22 стобцов и 23699 строк.\
Сразу мы можем обратить внимания на пропуски в следующих столбцах:\
высота потолков\
общее количество этажей\
жилая площадь\
является ли недвижимость - апартаментами\
площадь кухни\
количество балконов\
название населенного пункта\
расстояние до ближайшего аэропорта\
расстояние до ближайшего центра города\
кол-во парков в пределах 3 км\
кол-во водоемов в пределах 3 км\
расстояние до ближайшего парка\
расстояние до ближайшего водоема\
количество дней размещения объявления\
И обратить внимание, что в следующих столбцах неверные типы данных:
дата публикации\
общее количество этажей\
апартаменты\
кол-во балконов\
кол-во парков в пределах 3 км\
кол-во водоемов в пределах 3 км\
количество дней размещения объявления\
Необходимо обработать данные и рассмотреть каждый столбец в отдельности

### Предобработка данных

In [ ]:
old_shape = data.shape

Для дальнейшего сравнения какую долю информации мы сократили

**Исправляем названия столбцов:**

In [ ]:
data.columns
data = data.rename(
    columns={
        'cityCenters_nearest': 'city_centers_nearest',
        'parks_around3000': 'parks_around_3000',
        'ponds_around3000': 'ponds_around_3000',
    }
)

**Матрица корреляции:**

In [ ]:
data.corr()

**Заполняем пропущенные значения:**

In [ ]:
data['balcony'] = data['balcony'].fillna(0)
data['is_apartment'] = data['is_apartment'].fillna(False)
data['parks_around_3000'] = data['parks_around_3000'].fillna(0)
data['ponds_around_3000'] = data['ponds_around_3000'].fillna(0)
data['floors_total'] = data['floors_total'].fillna(data['floor'])
data['floors_total'] = data['floors_total'].fillna(1)
#data['days_exposition'] = data['days_exposition'].fillna(data['days_exposition'].max()) неверная логика
#data.loc[(data['floor'] == 1) & (data['floors_total'].isna())]

1) Если наличие балкона не указано, то будем считать, что его в квартире нет.\
2) В объявлении скорее всего отдельно указывается, что в предложении - апартаменты, или эту характеристику добавили позже даты публикацииобъявления. Указываем, что объявления с пропусками не про апартаменты.\
3,4) Парков и прудов рядом не имеется, недвижимость не в зеленой зоне.\
5) Если в объявлении не указана этажность здания, предположим, что дом или является одноэтажным (3 варианта) или указанный этаж и является последним ( было указано в описании объявления и не отобразилось при сборе данных)\
6)Предположим, что пропуски на месте пункта 'через сколько дней были сняты объявления' обусловлены тем, что на момент сбора информации предложение было актуально, заменим на максимальное значение в данном столбце(так как они относятся к долгосрочным объявлениям)

**Высота потолков:**

In [ ]:
#data['locality_name'].sort_values(ascending = True).unique()
rename_list = (
    ['посёлок городского типа ', 'поселок городского типа ', 'городской посёлок ', 'городской поселок ', 
               'поселок при железнодорожной станции ', 'поселок станции ', 'коттеджный поселок ', 'поселок ', 'посёлок ', 'деревня']
)
 
for substr in rename_list:
    data['locality_name'] = data['locality_name'].str.replace(substr, '')
data['locality_name'].unique()
for i in data['locality_name'].unique():
    data.loc[(data['locality_name'] == i) & (data['ceiling_height'].isna()), 'ceiling_height'] =\
    data.loc[(data['locality_name'] == i), 'ceiling_height'].median()

In [ ]:
data['ceiling_height'].isna().sum()

In [ ]:
data['ceiling_height'] = data['ceiling_height'].fillna(data['ceiling_height'].median())
#display(data['ceiling_height'].describe())
#data['ceiling_height'].median()

1)Попытка агрегации высоты потолков с населенными пунктами, в населенных пунктах застройка типовая( при условии большого количества малых населенных пунктов)\
2)Оставшиеся пропуски заменим медианным значением, аномально высокие значения не сильно повлияют\
**3) В данном пункте для агрегации заранее удалили неявные дубликаты из столбца 'locality_name'**

**Расстояние до центра города, аэропорта, парка, пруда:**

In [ ]:
for i in data['locality_name'].unique():
    data.loc[(data['locality_name'] == i) & (data['city_centers_nearest'].isna()), 'city_centers_nearest'] = \
    data.loc[(data['locality_name'] == i), 'city_centers_nearest'].median()
# for i in data['locality_name'].unique():
#     data.loc[(data['locality_name'] == i) & (data['airports_nearest'].isna()), 'airports_nearest'] = \
#     data.loc[(data['locality_name'] == i), 'airports_nearest'].median()
# for i in data['locality_name'].unique():
#     data.loc[(data['locality_name'] == i) & (data['parks_nearest'].isna()), 'parks_nearest'] = \
#     data.loc[(data['locality_name'] == i), 'parks_nearest'].median()
# for i in data['locality_name'].unique():
#     data.loc[(data['locality_name'] == i) & (data['ponds_nearest'].isna()), 'ponds_nearest'] = \
#     data.loc[(data['locality_name'] == i), 'ponds_nearest'].median()
#data['city_centers_nearest'].isna().sum()

In [ ]:
data['city_centers_nearest'].isna().sum()
data['airports_nearest'].isna().sum()

Агрегация этих параметров с населенными пунктами ожидаемого эффекта не принесла ( хоть и является грубой, но неожиданно)

In [ ]:
# data['city_centers_nearest'] = data['city_centers_nearest'].fillna(data['city_centers_nearest'].median())
# data['airports_nearest'] = data['airports_nearest'].fillna(data['airports_nearest'].median())
# data['parks_nearest'] = data['parks_nearest'].fillna(data['parks_nearest'].median())
# data['ponds_nearest'] = data['ponds_nearest'].fillna(data['ponds_nearest'].median())

In [ ]:
data['city_centers_nearest'].isna().sum()

Заменяем пропуски на медианное значение;\
Это очень грубое приближение, которое скажется на смещении значений;\
Было использовано только потому что слишком большой процент объявлений имеет пропуск, чтобы игнорировать

**Жилая площадь:**

In [ ]:
for i in data['rooms'].unique():
    data.loc[(data['rooms'] == i) & (data['living_area'].isna()), 'living_area'] =\
    data.loc[(data['rooms'] == i), 'living_area'].median()

In [ ]:
data['rooms'].isna().sum()

По таблице корреляции увидели достаточную зависимость между этими параметрами для агрегации (85%)\
Провели агрегацию между количеством комнат и жилой площадью и заполнили пропуски медианным значением.

**Площадь кухни:**

In [ ]:
#data['kitchen_area'].isna().sum()
#display(data['kitchen_area'].describe())
for i in data['open_plan'].unique():
    data.loc[(data['open_plan'] == True) & (data['kitchen_area'].isna()), 'kitchen_area'] =\
    data[ 'kitchen_area'].fillna(0)
for i in data['studio'].unique():
    data.loc[(data['studio'] == True) & (data['kitchen_area'].isna()), 'kitchen_area'] =\
    data[ 'kitchen_area'].fillna(0)  

In [ ]:
data['kitchen_area'].isna().sum()

Квартиры-студии и квартиры с планировкой по своему определению не имеют отдельной кухонной площади

In [ ]:
#data['kitchen_area'] = data['kitchen_area'].fillna(data['kitchen_area'].median())

In [ ]:
#data[(data['kitchen_area'] + data['living_area']) > data['total_area']].shape[0]

In [ ]:
data['kitchen_area'].isna().sum()

Заполняем оставшиеся пропуски медианным значением:\
Аномально высокие значения незначительно сказываются\
Основное количество значений держится не в таком широком диапазоне.

**Типы данных:**

In [ ]:
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format = '%Y-%m-%d')
data['is_apartment'] = data['is_apartment'].astype('bool')
data['balcony'] = data['balcony'].astype('int')
data['parks_around_3000'] = data['parks_around_3000'].astype('int')
data['ponds_around_3000'] = data['ponds_around_3000'].astype('int')
data['floors_total'] = data['floors_total'].astype('int')
#data['days_exposition'] = data['days_exposition'].astype('int')

**Удаление "аномалий"**

In [ ]:
#ceiling_height
def anomaly_change(height):
    if height > 19:
        return height / 10
    else:
        return height
data['ceiling_height'] = data.apply(lambda x: anomaly_change(x['ceiling_height']),axis=1)
# data['ceiling_height'] = data.loc[data['ceiling_height'] > 19, 'ceiling_height'] /= 10
display(data['ceiling_height'].describe())
data.boxplot('ceiling_height')
plt.ylim(0,8)
data.plot(y = 'ceiling_height', kind = 'hist', bins = 100, grid = True, range = (1,5))
plt.show()
#data.query('ceiling_height > 4')
#data.query('ceiling_height > 3.5')



In [ ]:

data.loc[data['ceiling_height'] > 19, 'ceiling_height'] /= 10

In [ ]:
data = data.loc[(data['ceiling_height'] >= 2.3) & (data['ceiling_height'] <= 4)]

Среднее значение высоты потолка: 2.7 м\
Аномальные значения высоты потолка больше 19 м мы обработали функцией, так как вероятно это ошибка порядка числа\
Ограничение провели в диапазоне высоты потолка от 2.3 м до 4 м \
Нижнее ограничение в соответсвии с требованиями СНиП\
Верхнее ограничение в соответствии с достаточным количеством объявлений, как вариант объявление о продаже 'сталинки'
В основном на рынке встречаются квартиры с высотой потолка от 2.4 до 3 м

In [ ]:
#data.query('floors_total == 60')
data.query('floors_total > 50')
def anomaly_change_1(floors_total):
    if floors_total > 55:
        return floors_total / 10
    else:
        return floors_total
data['floors_total'] = data.apply(lambda x: anomaly_change_1(x['floors_total']),axis=1)
# data = data.loc[data['floors_total'] > 50, 'floors_total'] /=10
display(data['floors_total'].describe())
data.boxplot('floors_total')
plt.ylim(0,70)
data.plot(y = 'floors_total', kind = 'hist', bins = 100, grid = True, range = (1,60))
plt.show()


In [ ]:
data= data.loc[(data['floors_total'] < 29) & (data['floors_total'] > 0)]

Среднее значение по общему количеству этажей - 10, медиана в свою очеред - 9\
Аномальное значение в 60 этажей, привели к 6 этажам ( первое предположение, что элитная высотка, но низкая стоимость и расположение в Кронштадте убедило, что ошибка в порядке числа)\
Ограничение провели в диапазоне до 29 этажей ( основная современная типовая застройка, да и выбросов выше достаточно мало)\
Основная масса предложений на малоэтажные дома в данной выборке

In [ ]:
display(data['rooms'].describe())
data.boxplot('rooms')
plt.ylim(0,20)
data.plot(y = 'rooms', kind = 'hist', bins = 100, grid = True, range = (1,20))
plt.show()

In [ ]:
#data.query('rooms > 8')
# data = data.loc[data['rooms'] < 8]

Среднее и медианное значение по количеству комнат в квартие - 2\
Мое предложение ограничить выборку до 8 комнат в квартире, больше это уже жилье мега элитного класса\
По графикам выше мы видим, что в основе своей рынок представлен небольшими квартирами (с 1 и 2 комнатами)\
А предложения с большим количество комнат во- первых малочисленны, во вторых могут влиять на среднюю цену в наших дальнейших исследованиях\
По нашим исследованиям рынок в-основном представлен 1-3 комнатными квартирами, где 3-комнатных квартир меньше.

In [ ]:
data.query('last_price < 100000')
def anomaly_change(price):
    if price < 100000:
        return price * 1000
    else:
        return price
data['last_price'] = data.apply(lambda x: anomaly_change(x['last_price']),axis=1)
display(data['last_price'].describe())
data.boxplot('last_price')
plt.ylim(500000,50000000)
data.plot(y = 'last_price', kind = 'hist', bins = 100, grid = True, range = (1,50000000))
plt.show()
#data[data['last_price'] > 12500000]
#data[data['last_price'] < 100000]

In [ ]:
#data = data.loc[data['last_price'] < 40000000 ]

Среднее значение по цене продажи квартиры - 5.5 миллиона рублей, а медианное - 4.5 миллиона\
Мое предложение ограничить выборку хотя бы с цена до 40 миллионов рублей, так как по графику основные прдложения находятся в категории до 10 миллионов (от 3 до 6), а среднее значение мы имеем на миллион больше значения медианы\
Аномальное значение в 12 тыс за квартиру мы обработали функцией, опираясь на данные этой квартиры\
Рынок по продаже квартир в основном представлен категорией до 10 миллионов рублей

In [ ]:
display(data['total_area'].describe())
data.boxplot('total_area')
plt.ylim(0,500)
data.plot(y = 'total_area', kind = 'hist', bins = 100, grid = True, range = (0, 150))
plt.show()

In [ ]:
data = data.loc[(data['total_area'] < 130) & (data['total_area'] > 18)]

Среднее значение по общей площади недвижимости - 55 квадратных метров, а медианное - 55 квадратов\
Ограничиваем выборку от 18 квадратов до 130:\
Нижние ограничение - СНиП, а верхнее - для уменьшения разброса\
Рынок представлен в основном, как было уже сказано 1-3 комнатными квартирами с общей площадью от 40 квадратов до 70 квадратов.

In [ ]:
display(data['living_area'].describe())
data.boxplot('living_area')
plt.ylim(0,200)
data.plot(y = 'living_area', kind = 'hist', bins = 100, grid = True, range = (0, 150))
plt.show()
#data.query('living_area > 150')


In [ ]:
data = data.loc[data['living_area'] > 8]

Среднее значение по жилой площади недвижимости - 31 квадратный метр, а медианное - 30 квадратов\
Ограничиваем выборку от 8 квадратов
Нижние ограничение -  требования СНиП
    Рынок представлен в основном, как было уже сказано 1-3 комнатными квартирами с жило площадью от 18 до 40 квадратов, основные пики на интервалах от 15 до 20 и от 25 до 35 ( изменение количества комнат в предложениях)

In [ ]:
display(data['floor'].describe())
data.boxplot('floor')
plt.ylim(0,29)
data.plot(y = 'floor', kind = 'hist', bins = 100, grid = True, range = (0, 29))
plt.show()

В основном рынок представлен квартирами, расположенными с 1 по 5 этаж ( как указывалось выше в основном продаются квартиры в малоэтажных зданиях)

In [ ]:
display(data['kitchen_area'].describe())
data.boxplot('kitchen_area')
plt.ylim(0,70)
data.plot(y = 'kitchen_area', kind = 'hist', bins = 100, grid = True, range = (0, 70))
plt.show()

Среднее значение по площади кухни - 10 квадратных метров, медианное - 9\
Наибольшее предложение на квартиры с площадью кухни от 7 до 11 квадратных метров.

In [ ]:
corr_shape = data.shape[0] / old_shape[0]
corr_shape # проверка на какой объем данных мы сократили выборку

### Посчитайте и добавьте в таблицу новые столбцы

**Цена одного квадратного метра:**

In [ ]:
data['price_per_square_meter'] = data['last_price'] / data['total_area']
data['price_per_square_meter'] = data['price_per_square_meter'].astype('int')
data['price_per_square_meter'] = data['price_per_square_meter'].round()
data[['price_per_square_meter', 'total_area', 'last_price']].head(10)


**День публикации объявления:**

In [ ]:
data['day_exposition'] = data['first_day_exposition'].dt.weekday
data[['first_day_exposition', 'day_exposition']].head(10)                                                 

**Месяц публикации объявления:**

In [ ]:
data['month_exposition'] = data['first_day_exposition'].dt.month
data[['first_day_exposition', 'month_exposition']].head(10)      

**Год публикации объявления:**

In [ ]:
data['year_exposition'] = data['first_day_exposition'].dt.year
data[['first_day_exposition', 'year_exposition']].head(10)    

**Этаж квартиры:**

In [ ]:
def floor_type(row):
    floor = row['floor']
    floors_total = row['floors_total'] 
    if floor == 1:
        return 'Первый'
    if 1< floor < floors_total:
        return 'Другой'
    else:
        return 'Последний'
data['floor_type'] = data.apply(floor_type, axis = 1)
data[['floor', 'floors_total', 'floor_type']].head(15)

**Расстояние до центра города в километрах:**

In [ ]:
data['city_centers_nearest_km'] = data['city_centers_nearest'] / 1000
# data['city_centers_nearest_km'] = data['city_centers_nearest_km'].astype('int')
# data['city_centers_nearest_km'] = data['city_centers_nearest_km'].round() использовал когда заменял пропуски медианой
data[['city_centers_nearest_km', 'city_centers_nearest']].head(10)

### Проведите исследовательский анализ данных

In [ ]:
display(data['floor_type'].describe())
data['floor_type'].hist()
plt.show()


In [ ]:
display(data['city_centers_nearest'].describe())
data.plot(y = 'city_centers_nearest', kind = 'hist', bins = 100, grid = True, range = (0, 60000))
plt.show()
#data.loc[(data['city_centers_nearest'] > 20000) & (data['city_centers_nearest'] < 25000)]['locality_name'].value_counts()


In [ ]:
display(data['airports_nearest'].describe())
data.plot(y = 'airports_nearest', kind = 'hist', bins = 100, grid = True, range = (0, 60000))
plt.show()

In [ ]:
display(data['ponds_nearest'].describe())
data.plot(y = 'ponds_nearest', kind = 'hist', bins = 100, grid = True, range = (0, 2000))
plt.show()

In [ ]:
display(data['parks_nearest'].describe())
data.plot(y = 'parks_nearest', kind = 'hist', bins = 100, grid = True, range = (0, 2000))
plt.show()

In [ ]:
data['day_exposition'].hist(bins=20)
plt.show()

In [ ]:
data['month_exposition'].hist(bins=20)
plt.show()

In [ ]:
display(data['total_area'].describe())
data.plot(y = 'total_area', kind = 'hist', bins = 100, grid = True, range = (0, 150))
plt.show()

In [ ]:
display(data['living_area'].describe())
data.plot(y = 'living_area', kind = 'hist', bins = 100, grid = True, range = (0, 150))
plt.show()

In [ ]:
display(data['kitchen_area'].describe())
data.plot(y = 'kitchen_area', kind = 'hist', bins = 100, grid = True, range = (0, 60))
plt.show()

In [ ]:
display(data['last_price'].describe())
data.plot(y = 'last_price', kind = 'hist', bins = 100, grid = True, range = (500000, 20000000))
plt.show()

In [ ]:
display(data['rooms'].describe())
data.plot(y = 'rooms', kind = 'hist', bins = 10, grid = True, range = (0, 15))
plt.show()

In [ ]:
display(data['ceiling_height'].describe())
data.plot(y = 'ceiling_height', kind = 'hist', bins = 30, grid = True, range = (0, 5))
plt.show()

In [ ]:
display(data['floor'].describe())
data.plot(y = 'floor', kind = 'hist', bins = 30, grid = True, range = (0, 30))
plt.show()

**Вывод:**
В данной выборке данных квартир на первом и последнем этаже - меньшинство\
Квартиры в основном находятся на расстоянии от 10 до 15 км от центра города, большое количество квартир на расстоянии 20 км от центра города, преимуществено Мурино и окраины Питера\
От аэропортов в свою очередь от 17 до 35 км\
Парки и пруды, учитывая то, что мы можем анализировать не все данные, находятся на расстоянии  от квартир в 300 - 600 метров( но при этом пруды имеют равномерное распределение, а парки в основном своем находятся в полукиллометре от квартиры)\
По дням недели мы можем сделать вывод, что люди занимаются размещением предложения на продажу квартир в рабочую неделю( в выходные - заметное снижение)\
В начале года пик объявлений на продажу квартир, затем приходится заметный спад на середину года и постепенное увеличение к концу года)



In [ ]:
display(data['days_exposition'].describe())
print('Медианное значение:', data['days_exposition'].median().round())
print('Среднее значение:', data['days_exposition'].mean().round())
data['days_exposition'].hist(bins =40)
plt.show()

Продажи квартиры за 1-1.5 месяца можно считать быстрыми\
Продажу квартиры от 1.5 до 3  месяцев можно считать нормальной\
Все остальное можно считать необычно долгой продажей ( опираясь на полученные данные и исследования Сбера)

 **Влияние на общую стоимость квартиры:**

In [ ]:
data['last_price_mln'] = data['last_price'] / 1000000
data['last_price_mln'] = data['last_price_mln'].astype('int')
data['last_price_mln'].round()

In [ ]:
pivot_table_total_area = data.pivot_table(index = 'total_area', values = 'last_price', aggfunc = ['median'])
pivot_table_total_area.plot(y = 'median', style = 'o')



data['total_area'].corr(data['last_price'])

Корелляция чуть чуть не дотягивает до хорошей, но мы видим явную прямую зависимость между этими двумя величинами

In [ ]:
pivot_table_total_area = data.pivot_table(index = 'living_area', values = 'last_price', aggfunc = ['median'])
pivot_table_total_area.plot(y = 'median', style = 'o')

data['living_area'].corr(data['last_price'])

С жилой площадью корреляция уже ниже, но все равно нормальная

In [ ]:
pivot_table_total_area = data.pivot_table(index = 'kitchen_area', values = 'last_price', aggfunc = ['median'])
pivot_table_total_area.plot(y = 'median', style = 'o')

data['kitchen_area'].corr(data['last_price'])

Примерно в таком же состоянии, что и зависимость выше, предположительно что увеличение размера кухни эти и увеличение жилой и общей площади, то есть квартира с большой кухней и маленькой жилой площадью это редкость.

In [ ]:
pivot_table_rooms = data.pivot_table(index = 'rooms', values = 'last_price', aggfunc = ['median'])
#display(pivot_table_rooms.head(10))
pivot_table_rooms.plot(y = 'median', style = '-o')

data['rooms'].corr(data['last_price'])

C увеличением числа комнат растет и стоимость,\

In [ ]:
pivot_table_floor_type = data.pivot_table(index = 'floor_type', values = 'last_price', aggfunc = ['median'])
pivot_table_floor_type.plot(y = 'median')
plt.show()


Квартиры на первом этаже - самые дешевые.

In [ ]:
pivot_table_day_exposition = data.pivot_table(index = 'day_exposition', values = 'last_price', aggfunc = ['mean', 'count', 'median'])
pivot_table_day_exposition.columns = ['mean', 'count', 'median']
pivot_table_day_exposition.plot(y = 'median')
pivot_table_day_exposition.sort_values('median', ascending = False)# проба более полно расписать сводную и впринципе анализ

In [ ]:
pivot_table_month_exposition = data.pivot_table(index = 'month_exposition', values = 'last_price', aggfunc = ['mean', 'count', 'median'])
pivot_table_month_exposition.columns = ['mean', 'count', 'median']
pivot_table_month_exposition.plot(y = 'median')
plt.title('Зависимость цены квартиры от месяца публикации ')
plt.xlabel('Месяц публикации')
plt.ylabel('Цена квартиры')
pivot_table_month_exposition.sort_values('median', ascending = False)

Явное проявление сезонности:\
Более дорогие квартиры выкладываются весной - в марте и апреле, летом - более дешевые \
Так и по количеству самое низкое предложение на "Новый Год" и лето ( отпуск)\
Предполагаю, что эта тенденция на рынке недвижимости сохраняется и для количества предложений и для цены, люди для продажи квартиры используют агентов, которые и дают советы по сезонности рынка.

In [ ]:
pivot_table_year_exposition = data.pivot_table(index = 'year_exposition', values = 'last_price', aggfunc = ['mean', 'count', 'median'])
pivot_table_year_exposition.columns = ['mean', 'count', 'median']
pivot_table_year_exposition.plot(y = 'median')
pivot_table_year_exposition.sort_values('median', ascending = False)

Предположение, что политические события в мире в 2014 году образовывают желание более состоятельной части жителей Ленинградской области покинуть страну, для чего и продается более дорогая недвижимость.

In [ ]:
locality_name_pivot_table = data.pivot_table(index = 'locality_name', values = 'price_per_square_meter', aggfunc=['count', 'mean'])
locality_name_pivot_table.columns = ['count', 'mean']
# средняя цена квадратного метра в 10 населенных пунктах с наибольшим числом объявлений 
up_10_locality_name_pivot_table = locality_name_pivot_table.sort_values('count', ascending = False).head(10)
up_10_locality_name_pivot_table[up_10_locality_name_pivot_table['mean']==up_10_locality_name_pivot_table['mean'].max()]
up_10_locality_name_pivot_table[up_10_locality_name_pivot_table['mean']==up_10_locality_name_pivot_table['mean'].min()]
up_10_locality_name_pivot_table

Самая высокая цена за квадратный метр в столице области ( Санкт - Петербург) - 111451 рублей\
Самая низкая цена за квадратный метр в Выборге - 58208 рублей

In [ ]:
#city_centers_nearest_km
# нужно округление 
data['city_centers_nearest_km'] = data['city_centers_nearest_km'].fillna(1000)
data['city_centers_nearest_km'] = data['city_centers_nearest_km'].astype('int')
data['city_centers_nearest_km'] = data['city_centers_nearest_km'].round()
# pivot_table_city_centers = data.query('locality_name == "Санкт-Петербург" and city_centers_nearest.isna() != True').pivot_table(index = 'city_centers_nearest_km', values = 'last_price', aggfunc = ['mean'])
pivot_table_city_centers = data.query('locality_name == "Санкт-Петербург" and city_centers_nearest.isna() != 1000').pivot_table(index = 'city_centers_nearest_km', values = 'last_price', aggfunc = ['mean'])

pivot_table_city_centers.plot()

plt.show()

pivot_table_city_centers

С удалением от центра города стоимость недвижимости снижается

### Общий вывод

1)Была произведена обработка полученных данных:\
Заполнены пропуски;\
Преобразовали типы данных в столбцах;\
Устранили неявные дубликаты в столбце 'locality_name';\
Нашли и устранили редкие и выбивающиеся значения из столбцов.\
2)Добавили в датафрейм следующие данные по квартирам:\
цена одного квадратного метра;\
день недели публикации объявления;
месяц публикации объявления;\
год публикации объявления;\
тип этажа квартиры;\
расстояние до центра города в километрах.\
3)Изучили какие квартиры составляют основу рынка предложения:\
Это 1-2(3) комнатные квартиры в малоэтажных домах, с общей площадью от 40 до 70 квадратных метров (жилая 18-40), с типовой высотой этажей от 2.5 до 3 м.\
Цена на такие квартиры колеблется от 3.5 до 6.5 миллионов рублей\
Квартир на первом и последнем этаже - меньшинство\
Квартиры в основном находятся на расстоянии от 10 до 15 км от центра города, большое количество квартир на расстоянии 20 км от центра города, преимуществено Мурино и окраины Питера\
От аэропортов в свою очередь от 17 до 35 км\
Парки и пруды находятся в основном на равном удалении от квартир от 300 до 600 метров\
По дням недели мы можем сделать вывод, что люди занимаются размещением предложения на продажу квартир в рабочую неделю( в выходные - заметное снижение)\
В начале года пик объявлений на продажу квартир, затем приходится заметный спад на середину года и постепенное увеличение к концу года)\
4)Основываясь на диаграмме быстрая продажа квартиры - до 1.5 месяцев, нормальная продажа квартиры до 3 месяцев, остальное необычно долгая продажа\
5)На общую стоимость квартиры влияет общая площадь, жилая площадь, площадь кухни, количество комнат\
Квартиры на первом этаже - самые дешевые\
День и месяц незначительно влияют на среднюю цену квартир, сильная динамика в ценаъ проявляется только в 2014-2015 годах.\
6)Самая низкая цена за квадратный метр в Выборге, самая высокая соответственно в Санкт-Петербурге